In [9]:
import pandas as pd
from pathlib import Path
import datetime as dt

In [3]:
PROJPATH = Path().resolve().parent
breakupdata = PROJPATH / "data/breakupdata/derived"
stationdata = PROJPATH / "data/weatherstations/ACIS/TDD/tdd_cumul_bystation"


In [6]:
breakuppth = breakupdata / "breakupDate_cleaned.csv"
breakup = pd.read_csv(breakuppth, header=3, index_col=0)
breakup 

,id,siteID,year,breakup
0,7175,Koyukuk River at Bettles,2022,2022-05-05
1,7173,Kobuk River at Shungnak,2022,2022-05-20
2,7172,Buckland River at Buckland,2022,2022-05-21
3,7171,Yukon River at Mountain Village,2022,2022-05-10
4,7170,Yukon River at Marshall,2022,2022-05-15
...,...,...,...,...
1490,213,Buckland River at Buckland,1988,1988-05-05
1491,212,Buckland River at Buckland,1987,1987-05-22
1492,211,Buckland River at Buckland,1986,1986-05-30
1493,210,Buckland River at Buckland,1984,1984-05-23


In [7]:
stationdata

PosixPath('/Users/chris/Projects/2023_Riverice/data/weatherstations/ACIS/TDD/tdd_cumul_bystation')

In [8]:
tdd = sorted(list(stationdata.glob("*.csv")))
tdd

[PosixPath('/Users/chris/Projects/2023_Riverice/data/weatherstations/ACIS/TDD/tdd_cumul_bystation/ALYESKA_yearly_TDD_cumul.csv'),
 PosixPath('/Users/chris/Projects/2023_Riverice/data/weatherstations/ACIS/TDD/tdd_cumul_bystation/ANCHORAGE_MERRILL_FIELD_yearly_TDD_cumul.csv'),
 PosixPath('/Users/chris/Projects/2023_Riverice/data/weatherstations/ACIS/TDD/tdd_cumul_bystation/ANCHORAGE_TED_STEVENS_INTERNATIONAL_AIRPORT_yearly_TDD_cumul.csv'),
 PosixPath('/Users/chris/Projects/2023_Riverice/data/weatherstations/ACIS/TDD/tdd_cumul_bystation/AUKE_BAY_yearly_TDD_cumul.csv'),
 PosixPath('/Users/chris/Projects/2023_Riverice/data/weatherstations/ACIS/TDD/tdd_cumul_bystation/BARROW_AIRPORT_yearly_TDD_cumul.csv'),
 PosixPath('/Users/chris/Projects/2023_Riverice/data/weatherstations/ACIS/TDD/tdd_cumul_bystation/BETHEL_AIRPORT_yearly_TDD_cumul.csv'),
 PosixPath('/Users/chris/Projects/2023_Riverice/data/weatherstations/ACIS/TDD/tdd_cumul_bystation/BETTLES_AIRPORT_yearly_TDD_cumul.csv'),
 PosixPath('/Us

In [16]:
(dt.datetime.strptime("2022-05-05", "%Y-%m-%d").date() - dt.date(2022, 3, 1)).days

65

In [33]:
with open(outpath / "breakupDate_cleaned.csv", "w") as dst:
    dst.write("# Data from NWS-APRFC_AK_BreakupDate.csv\n")
    dst.write("# Only sites with more than 30 data points since 1980\n")
    dst.write("# \n")
    breakup_clean.to_csv(dst)    